In [29]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()

In [30]:
dataset = pd.read_csv('Churn_Modelling.csv', index_col = 'RowNumber')
dataset.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,,,,
1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [31]:
#Customer ID and Surname would not be relevant as features
X_columns = dataset.columns.tolist()[2:12]
Y_columns = dataset.columns.tolist()[-1:]
print(X_columns)
print(Y_columns)

['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
['Exited']


In [32]:
X = dataset[X_columns].values 
Y = dataset[Y_columns].values

In [33]:
#We need to encode categorical variables such as geography and gender
from sklearn.preprocessing import LabelEncoder
X_column_transformer = LabelEncoder()
X[:, 1] = X_column_transformer.fit_transform(X[:, 1])

In [34]:
#Lets Encode gender now
X[:, 2] = X_column_transformer.fit_transform(X[:, 2])

In [35]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

pipeline = Pipeline(
    [
        ('Categorizer', ColumnTransformer(
            [
                ("Gender Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [2]),
                ("Geography Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [1])
            ], 
            remainder = 'passthrough', n_jobs = 1)),
        ('Normalizer', StandardScaler())
    ]
)

In [36]:
#Standardize the features
X = pipeline.fit_transform(X)

In [37]:
#Spilt the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)
#Let us create the Neural Network
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [38]:
#Initialize ANN
classifier = Sequential()
#Add input layer and hidden layer
classifier.add(Dense(6, activation = 'relu', input_shape = (X_train.shape[1], )))
classifier.add(Dropout(rate = 0.1))
#Add second layer
classifier.add(Dense(6, activation = 'relu'))
classifier.add(Dropout(rate = 0.1))
#Add output layer
classifier.add(Dense(1, activation = 'sigmoid'))
#Let us take a look at our network
classifier.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 6)                 72        
                                                                 
 dropout_4 (Dropout)         (None, 6)                 0         
                                                                 
 dense_7 (Dense)             (None, 6)                 42        
                                                                 
 dropout_5 (Dropout)         (None, 6)                 0         
                                                                 
 dense_8 (Dense)             (None, 1)                 7         
                                                                 
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [39]:
#Optimize the weights
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#Fitting the Neural Network
history = classifier.fit(X_train, y_train, batch_size = 32, epochs = 200, validation_split = 0.1, verbose = 2)
y_pred = classifier.predict(X_test)
print(y_pred[:5])

Epoch 1/200
225/225 - 1s - loss: 0.6083 - accuracy: 0.7254 - val_loss: 0.4801 - val_accuracy: 0.7937 - 972ms/epoch - 4ms/step
Epoch 2/200
225/225 - 0s - loss: 0.4731 - accuracy: 0.8001 - val_loss: 0.4426 - val_accuracy: 0.8000 - 363ms/epoch - 2ms/step
Epoch 3/200
225/225 - 0s - loss: 0.4566 - accuracy: 0.8043 - val_loss: 0.4309 - val_accuracy: 0.8125 - 292ms/epoch - 1ms/step
Epoch 4/200
225/225 - 0s - loss: 0.4499 - accuracy: 0.8053 - val_loss: 0.4229 - val_accuracy: 0.8150 - 285ms/epoch - 1ms/step
Epoch 5/200
225/225 - 0s - loss: 0.4477 - accuracy: 0.8065 - val_loss: 0.4185 - val_accuracy: 0.8175 - 301ms/epoch - 1ms/step
Epoch 6/200
225/225 - 0s - loss: 0.4403 - accuracy: 0.8069 - val_loss: 0.4142 - val_accuracy: 0.8238 - 303ms/epoch - 1ms/step
Epoch 7/200
225/225 - 0s - loss: 0.4404 - accuracy: 0.8079 - val_loss: 0.4121 - val_accuracy: 0.8238 - 282ms/epoch - 1ms/step
Epoch 8/200
225/225 - 0s - loss: 0.4361 - accuracy: 0.8118 - val_loss: 0.4085 - val_accuracy: 0.8275 - 282ms/epoch - 1

Epoch 66/200
225/225 - 0s - loss: 0.4084 - accuracy: 0.8308 - val_loss: 0.3731 - val_accuracy: 0.8487 - 274ms/epoch - 1ms/step
Epoch 67/200
225/225 - 0s - loss: 0.4090 - accuracy: 0.8306 - val_loss: 0.3720 - val_accuracy: 0.8537 - 273ms/epoch - 1ms/step
Epoch 68/200
225/225 - 0s - loss: 0.4091 - accuracy: 0.8301 - val_loss: 0.3714 - val_accuracy: 0.8550 - 272ms/epoch - 1ms/step
Epoch 69/200
225/225 - 0s - loss: 0.4118 - accuracy: 0.8294 - val_loss: 0.3727 - val_accuracy: 0.8487 - 282ms/epoch - 1ms/step
Epoch 70/200
225/225 - 0s - loss: 0.4102 - accuracy: 0.8304 - val_loss: 0.3717 - val_accuracy: 0.8525 - 273ms/epoch - 1ms/step
Epoch 71/200
225/225 - 0s - loss: 0.4086 - accuracy: 0.8294 - val_loss: 0.3714 - val_accuracy: 0.8500 - 273ms/epoch - 1ms/step
Epoch 72/200
225/225 - 0s - loss: 0.4117 - accuracy: 0.8311 - val_loss: 0.3723 - val_accuracy: 0.8500 - 272ms/epoch - 1ms/step
Epoch 73/200
225/225 - 0s - loss: 0.4090 - accuracy: 0.8328 - val_loss: 0.3707 - val_accuracy: 0.8500 - 276ms/e

Epoch 131/200
225/225 - 0s - loss: 0.3910 - accuracy: 0.8247 - val_loss: 0.3491 - val_accuracy: 0.8438 - 273ms/epoch - 1ms/step
Epoch 132/200
225/225 - 0s - loss: 0.3860 - accuracy: 0.8283 - val_loss: 0.3484 - val_accuracy: 0.8462 - 274ms/epoch - 1ms/step
Epoch 133/200
225/225 - 0s - loss: 0.3860 - accuracy: 0.8257 - val_loss: 0.3478 - val_accuracy: 0.8450 - 272ms/epoch - 1ms/step
Epoch 134/200
225/225 - 0s - loss: 0.3880 - accuracy: 0.8246 - val_loss: 0.3488 - val_accuracy: 0.8475 - 271ms/epoch - 1ms/step
Epoch 135/200
225/225 - 0s - loss: 0.3837 - accuracy: 0.8278 - val_loss: 0.3473 - val_accuracy: 0.8500 - 276ms/epoch - 1ms/step
Epoch 136/200
225/225 - 0s - loss: 0.3856 - accuracy: 0.8268 - val_loss: 0.3459 - val_accuracy: 0.8462 - 272ms/epoch - 1ms/step
Epoch 137/200
225/225 - 0s - loss: 0.3867 - accuracy: 0.8271 - val_loss: 0.3466 - val_accuracy: 0.8475 - 273ms/epoch - 1ms/step
Epoch 138/200
225/225 - 0s - loss: 0.3821 - accuracy: 0.8288 - val_loss: 0.3451 - val_accuracy: 0.8450 -

225/225 - 0s - loss: 0.3780 - accuracy: 0.8514 - val_loss: 0.3315 - val_accuracy: 0.8687 - 271ms/epoch - 1ms/step
Epoch 196/200
225/225 - 0s - loss: 0.3754 - accuracy: 0.8543 - val_loss: 0.3303 - val_accuracy: 0.8675 - 275ms/epoch - 1ms/step
Epoch 197/200
225/225 - 0s - loss: 0.3716 - accuracy: 0.8517 - val_loss: 0.3303 - val_accuracy: 0.8600 - 271ms/epoch - 1ms/step
Epoch 198/200
225/225 - 0s - loss: 0.3771 - accuracy: 0.8504 - val_loss: 0.3312 - val_accuracy: 0.8700 - 273ms/epoch - 1ms/step
Epoch 199/200
225/225 - 0s - loss: 0.3736 - accuracy: 0.8515 - val_loss: 0.3304 - val_accuracy: 0.8712 - 276ms/epoch - 1ms/step
Epoch 200/200
225/225 - 0s - loss: 0.3737 - accuracy: 0.8524 - val_loss: 0.3304 - val_accuracy: 0.8687 - 271ms/epoch - 1ms/step
63/63 [==============================] - 0s 1ms/step
[[0.2605697 ]
 [0.36872467]
 [0.19709434]
 [0.08701656]
 [0.11972681]]


In [40]:
#Let us use confusion matrix with cutoff value as 0.5
y_pred = (y_pred > 0.5).astype(int)
print(y_pred[:5])

[[0]
 [0]
 [0]
 [0]
 [0]]


In [41]:
#Making the Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1523   72]
 [ 195  210]]


In [42]:
#Accuracy of our NN
print(((cm[0][0] + cm[1][1])* 100) / len(y_test), '% of data was classified correctly')

86.65 % of data was classified correctly
